<a href="https://colab.research.google.com/github/SubMishMar/rotation_averaging/blob/main/single_rotation_averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [64]:
np.random.seed(42)  # reproducibility

In [65]:
# Utility

def log_angle(R):
    t = (np.trace(R) - 1.0)*0.5
    t = np.clip(t, -1.0, 1.0)
    return float(np.arccos(t))

def ang_error_deg(Ra, Rb):
    return np.degrees(log_angle(Ra.T @ Rb))

def add_rotation_noise(R_true, noise_level_deg=5.0):
    axis = np.random.randn(3)
    axis /= np.linalg.norm(axis)
    angle = np.random.normal(0, np.deg2rad(noise_level_deg))
    R_noise = R.from_rotvec(axis * angle).as_matrix()
    return R_true.as_matrix() @ R_noise


def hat(w):
    wx, wy, wz = w
    return np.array([[   0, -wz,  wy],
                     [  wz,   0, -wx],
                     [ -wy,  wx,   0]], dtype=float)

def vee(S):
    return np.array([S[2,1], S[0,2], S[1,0]], dtype=float)

def exponential_map(w, eps=1e-8):
    theta = np.linalg.norm(w)
    W = hat(w)
    if theta < eps:
        # A ~ 1 - θ^2/6, B ~ 1/2 - θ^2/24
        A = 1.0 - (theta**2)/6.0
        B = 0.5 - (theta**2)/24.0
    else:
        A = np.sin(theta) / theta
        B = (1.0 - np.cos(theta)) / (theta**2)
    return np.eye(3) + A * W + B * (W @ W)


def logarithm_map(R, eps=1e-8):
    tr = np.trace(R)
    cos_phi = (tr - 1.0) * 0.5
    cos_phi = np.clip(cos_phi, -1.0, 1.0)
    phi = np.arccos(cos_phi)

    if phi < eps:
        return 0.5 * vee(R - R.T)

    if np.pi - phi < 1e-5:
        Rp = (R + np.eye(3)) * 0.5
        idx = np.argmax(np.diag(Rp))
        v = np.zeros(3)
        v[idx] = np.sqrt(max(Rp[idx, idx], 0.0))
        j = (idx + 1) % 3
        k = (idx + 2) % 3
        if v[idx] > eps:
            v[j] = Rp[j, idx] / v[idx]
            v[k] = Rp[k, idx] / v[idx]
        v = v / (np.linalg.norm(v) + eps)
        return phi * v

    # Generic case
    return (phi / (2.0 * np.sin(phi))) * vee(R - R.T)

def right_jacobian(v, eps=1e-8):
    theta = np.linalg.norm(v)
    V = hat(v)
    if theta < eps:
        return np.eye(3) - 0.5*V + (1.0/12.0)*(V @ V)
    A = (1.0 - np.cos(theta)) / (theta**2)
    B = (theta - np.sin(theta)) / (theta**3)
    return np.eye(3) - A*V + B*(V @ V)

def right_jacobian_inverse(v, eps=1e-8):
    theta = np.linalg.norm(v)
    V = hat(v)
    if theta < eps:
        return np.eye(3) + 0.5*V + (1.0/12.0)*(V @ V)
    cot_half = 1.0 / np.tan(0.5 * theta)
    B = (1.0 / (theta**2)) - (0.5 / theta) * cot_half
    return np.eye(3) + 0.5*V + B*(V @ V)

def get_G():
    G = np.zeros((9,3))
    # column-stacked vec; hat([dx,dy,dz]) columns: [ [0, dz, -dy], [-dz, 0, dx], [dy, -dx, 0] ]
    G[1,2] =  1.0; G[2,1] = -1.0
    G[3,2] = -1.0; G[5,0] =  1.0
    G[6,1] =  1.0; G[7,0] = -1.0
    return G

def cost(residual):
    return 0.5 * (residual.T @ residual).item()

def project_to_so3(M):
    U,_,Vt = np.linalg.svd(M)
    R = U@Vt
    if np.linalg.det(R) < 0:
        U[:,-1]*=-1
        R = U@Vt
    return R

def vec_col(A):
    return A.reshape(-1, order='F')

In [66]:
# Chordal Averaging
def chordal_average(rotation_matrices):
    # Computes the chordal average of a set of rotation matrices.
    # This is the minimizer when rotations are assumed to be drawn from Langevin
    # Distribution.

    # Step 1: Average the rotation matrices.
    M = np.zeros((3, 3))
    for Rn in rotation_matrices:
        M += Rn
    M /= len(rotation_matrices)
    # Step 2: Project the average matrix onto SO(3).
    U, _, Vt = np.linalg.svd(M)
    R = U @ Vt
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = U @ Vt
    return R

In [67]:
# Minimize Chordal difference using NLLS
# Minimization of Chordal difference results when rotations are assumed to be
# sampled from a Langevin distribution over Lie Groups
def residuals_and_jacobians_chordal_difference(R_est, R_meas_list):
    rows = []
    jacobians = []
    jacobian_i = np.kron(np.eye(3), R_est) @ get_G()
    for Rm in R_meas_list:
        r_i = vec_col(R_est - Rm)
        rows.append(r_i[:,None])
        jacobians.append(jacobian_i)

    return np.vstack(rows), np.vstack(jacobians)

In [68]:
# Minimize Geodesic difference NLLS
# Minimization of Geodesic difference results when rotations are assumed to be
# sampled from a wrapped Gaussian distribution over Lie groups
def residuals_and_jacobians_log_error(R_est, R_meas_list):
    rows = []
    jacobians = []
    Rt = R_est.T
    for Rm in R_meas_list:
        r_i = logarithm_map(Rt @ Rm)
        jacobian_i = -right_jacobian_inverse(r_i) @ Rm.T @ R_est
        rows.append(r_i.reshape(-1, 1))
        jacobians.append(jacobian_i)
    return np.vstack(rows), np.vstack(jacobians)

In [69]:
# Solver
def gauss_newton(R_meas_list, R_init, max_iters=100, f_tol=1e-10, x_tol=1e-12,
                 method='chordal', lm_lambda=0.0):
    R_est = R_init.copy()
    history = []
    for k in range(max_iters):
        if method == 'chordal':
            r, J = residuals_and_jacobians_chordal_difference(R_est, R_meas_list)
        else:
            r, J = residuals_and_jacobians_log_error(R_est, R_meas_list)

        f = cost(r)
        H = J.T @ J
        g = J.T @ r
        if lm_lambda > 0.0:
            H = H + lm_lambda * np.eye(3)

        delta = (-np.linalg.inv(H) @ g).reshape(3)
        R_new = project_to_so3(R_est @ exponential_map(delta))

        if method == 'chordal':
            r_new, _ = residuals_and_jacobians_chordal_difference(R_new, R_meas_list)
        else:
            r_new, _ = residuals_and_jacobians_log_error(R_new, R_meas_list)

        f_new = cost(r_new)
        step = float(np.linalg.norm(delta))
        history.append((k, f_new, step))

        R_est = R_new

        if abs(f_new - f) < f_tol or step < x_tol:
            break
    return R_est, history

In [70]:
# Generate a True Rotation Value
angle_x = 30.0  # degrees
angle_y = 20.0
angle_z = 10.0
euler_order = "xyz"
true_euler_deg = {
    'x': angle_x,
    'y': angle_y,
    'z': angle_z
}
angles = [true_euler_deg[axis] for axis in euler_order]
R_true = R.from_euler(euler_order, angles, degrees=True)

# Generate N noisy versions
N = 10
noisy_rotations = [add_rotation_noise(R_true, noise_level_deg=2.5) for _ in range(N)]
R_init = add_rotation_noise(R_true, noise_level_deg=50.0)
for i, Rn in enumerate(noisy_rotations):
  rot_vec = R.from_matrix(Rn).as_euler('xyz', degrees=True)
  #print(f"\nNoisy Rotation {i+1}:\n{rot_vec}")

In [71]:
R_est_chordal_avg = chordal_average(noisy_rotations)
print('\nChordal Averaging\n')
print('\nR_true: \n', R_true.as_euler('xyz', degrees=True))
print('R_init: \n', R.from_matrix(R_init).as_euler('xyz', degrees=True))
print('R_estimated: \n', R.from_matrix(R_est_chordal_avg).as_euler('xyz', degrees=True))


Chordal Averaging


R_true: 
 [30. 20. 10.]
R_init: 
 [15.75059527 16.02822191 10.82211481]
R_estimated: 
 [29.88637802 19.85218462 10.21233337]


In [72]:
R_est_geodesic_gn, history = gauss_newton(noisy_rotations, R_init)
print('\nGeodesic GN\n')
print('\nR_true: \n', R_true.as_euler('xyz', degrees=True))
print('R_init: \n', R.from_matrix(R_init).as_euler('xyz', degrees=True))
print('R_estimated: \n', R.from_matrix(R_est_geodesic_gn).as_euler('xyz', degrees=True))


Geodesic GN


R_true: 
 [30. 20. 10.]
R_init: 
 [15.75059527 16.02822191 10.82211481]
R_estimated: 
 [29.88637789 19.85218463 10.21233333]


In [73]:
R_est_chordal_gn, history = gauss_newton(noisy_rotations, R_init, method="chordal")
print('\nChordal GN\n')
print('\nR_true: \n', R_true.as_euler('xyz', degrees=True))
print('R_init: \n', R.from_matrix(R_init).as_euler('xyz', degrees=True))
print('R_estimated: \n', R.from_matrix(R_est_chordal_gn).as_euler('xyz', degrees=True))


Chordal GN


R_true: 
 [30. 20. 10.]
R_init: 
 [15.75059527 16.02822191 10.82211481]
R_estimated: 
 [29.88637789 19.85218463 10.21233333]


In [74]:
# sanity checks
print("Errors to truth (deg):")
print(f"  Chordal Avg.  : {ang_error_deg(R_true.as_matrix(), R_est_chordal_avg):.4f}")
print(f"  Geodesic GN   : {ang_error_deg(R_true.as_matrix(), R_est_geodesic_gn):.4f}")
print(f"  Chordal GN: {ang_error_deg(R_true.as_matrix(), R_est_chordal_gn):.4f}")

Errors to truth (deg):
  Chordal Avg.  : 0.3103
  Geodesic GN   : 0.3103
  Chordal GN: 0.3103
